## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy
duet1 = sy.duet("de12b5c540d4b2e5464fcf08e40a210b")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: dd1c3bcf32c4e528f8d9f38f2a41070d

♫♫♫ > ...waiting for partner to connect...


C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [2]:
duet2 = sy.duet("074b43b0b1cd328cc18e50d4b0bf7353")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 92996a0e22b3bfb3552722a78a60e881

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [3]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 47e9e8d2f9814ae09139d6096f1cb412>,[DO1 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: 9c0d5bc308174fda91f4c182ba915f84>,[DO1 training y data],number of samples and features,<class 'torch.Tensor'>


In [4]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: e9cd85a51bc348479d8fdbccfd104e01>,[DO2 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: e8a06f7f0d0a40b38fe568d075f1bb97>,[DO2 training y data],number of samples and features,<class 'torch.Tensor'>


## Modelling

In [5]:
data1_ptr = duet1.store[0]
target1_ptr = duet1.store[1]
data2_ptr = duet2.store[0]
target2_ptr = duet2.store[1]

print(data1_ptr)
print(target1_ptr)
print(data2_ptr)
print(target2_ptr)

### Create Base Model

In [6]:
import torch
import torch.nn.functional as F
import torch.nn as nn 
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [7]:
#in_dim = 39
#out_dim = 1

In [8]:
#class SyNet(sy.Module):
    #def __init__(self, torch_ref):
        #super(SyNet, self).__init__(torch_ref=torch_ref)
        #self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    #def forward(self, x):
        #x = self.linear(x)
        #return x

In [9]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.layer_1 = self.torch_ref.nn.Linear(39, 31)
        self.layer_out = self.torch_ref.nn.Linear(31, 1)
        # Define proportion or neurons to dropout
        self.dropout_1 = self.torch_ref.nn.Dropout(0.12409392594394411)
        self.relu = self.torch_ref.nn.ReLU()
    
    def forward(self, inputs):
        x = self.dropout_1(self.relu(self.layer_1(inputs)))
        x = self.layer_out(x)

        return (x)

    def predict(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.layer_out(x)

        return (x)

### Training

In [10]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    #criterion = torch_ref.nn.functional.poisson_nll_loss(log_input= True, full= True)

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        #loss = torch_ref.nn.functional.nll_loss(output, target_ptr)
        loss = torch_ref.nn.functional.poisson_nll_loss(output, target_ptr, log_input= True, full= True)   
        #loss = torch_ref.nn.functional.mse_loss(output, target_ptr)
        #loss = criterion(output, target_ptr)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses

#### Send one copy of the model to each data owner or client and train them remotely one by one

In [11]:
import torch as th
import numpy as np

In [12]:
base_model = SyNet(torch)

Train on Data Owner 1 data

In [13]:
remote_model1 = base_model.send(duet1)

In [14]:
remote_torch1 = duet1.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

In [15]:
iteration = 25
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

Epoch 0 loss 1.1209511756896973
Epoch 1 loss 0.25860995054244995
Epoch 2 loss 0.3237025737762451
Epoch 3 loss 0.3992210924625397
Epoch 4 loss 0.3737008571624756
Epoch 5 loss 0.3037089407444
Epoch 6 loss 0.238212451338768
Epoch 7 loss 0.2220996916294098
Epoch 8 loss 0.252182275056839
Epoch 9 loss 0.26240459084510803
Epoch 10 loss 0.24415290355682373
Epoch 11 loss 0.22806397080421448
Epoch 12 loss 0.22489392757415771
Epoch 13 loss 0.2301066368818283
Epoch 14 loss 0.23699286580085754
Epoch 15 loss 0.23910272121429443
Epoch 16 loss 0.2371058464050293
Epoch 17 loss 0.23202422261238098
Epoch 18 loss 0.22562023997306824
Epoch 19 loss 0.22114117443561554
Epoch 20 loss 0.22090882062911987
Epoch 21 loss 0.2237473726272583
Epoch 22 loss 0.22645746171474457
Epoch 23 loss 0.2257240116596222
Epoch 24 loss 0.2229830026626587


Train on Data Owner 2 data

In [16]:
remote_model2 = base_model.send(duet2)

In [17]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

In [18]:
iteration = 25
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 1.121098279953003
Epoch 1 loss 0.25986993312835693
Epoch 2 loss 0.32722070813179016
Epoch 3 loss 0.4022490680217743
Epoch 4 loss 0.37463411688804626
Epoch 5 loss 0.3028140366077423
Epoch 6 loss 0.23840069770812988
Epoch 7 loss 0.22499756515026093
Epoch 8 loss 0.25599631667137146
Epoch 9 loss 0.2645457983016968
Epoch 10 loss 0.24533917009830475
Epoch 11 loss 0.22992955148220062
Epoch 12 loss 0.22758378088474274
Epoch 13 loss 0.2325691133737564
Epoch 14 loss 0.23885966837406158
Epoch 15 loss 0.2417428344488144
Epoch 16 loss 0.24010485410690308
Epoch 17 loss 0.23455728590488434
Epoch 18 loss 0.22784149646759033
Epoch 19 loss 0.2233542650938034
Epoch 20 loss 0.2233262062072754
Epoch 21 loss 0.22597098350524902
Epoch 22 loss 0.22887744009494781
Epoch 23 loss 0.22832833230495453
Epoch 24 loss 0.22498919069766998


### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [19]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[-0.1211, -0.1358,  0.0471,  ..., -0.1219, -0.1267,  0.1028],
        [-0.0649,  0.1530,  0.0503,  ..., -0.0047,  0.0957,  0.1152],
        [-0.1276,  0.0625, -0.1195,  ..., -0.1430, -0.0908, -0.0712],
        ...,
        [-0.0806,  0.0594, -0.0034,  ..., -0.1556,  0.0472, -0.0163],
        [-0.0803,  0.0289, -0.0450,  ...,  0.0862, -0.1593,  0.0128],
        [ 0.1295,  0.0422, -0.0792,  ..., -0.0011, -0.1172,  0.0231]],
       requires_grad=True), Parameter containing:
tensor([-0.0326,  0.0418,  0.0228, -0.0239,  0.0411, -0.0999,  0.0229, -0.0373,
        -0.0378,  0.0297,  0.0959,  0.0957,  0.0177, -0.1532, -0.1282,  0.0144,
        -0.0893, -0.0959,  0.1048,  0.0772,  0.0933,  0.1107,  0.1305, -0.1339,
         0.0184,  0.0191,  0.0554, -0.1130,  0.1571,  0.1201, -0.0197],
       requires_grad=True), Parameter containing:
tensor([[ 0.0698,  0.1080, -0.0232, -0.0243,  0.0024, -0.0231, -0.0880, -0.1620,
          0.0394, -0.0583, 

As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [20]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

OrderedDict([('layer_1.weight', tensor([[-0.6777, -0.6924, -0.5095,  ..., -0.1219, -0.6646, -0.4533],
        [-0.6215, -0.4037, -0.5063,  ..., -0.5613, -0.4609, -0.4414],
        [-0.3288,  0.1863,  0.0653,  ...,  0.3790,  0.2414,  0.1923],
        ...,
        [ 0.0985,  0.3905,  0.3582,  ...,  0.3909,  0.4855,  0.4172],
        [-0.6369, -0.5278, -0.6016,  ..., -0.4705, -0.7159, -0.5438],
        [-0.4271, -0.5144, -0.6358,  ..., -0.5577, -0.6738, -0.5335]])), ('layer_1.bias', tensor([-0.5892, -0.5148,  0.3019,  0.0704, -0.5155, -0.4367,  0.1437, -0.2604,
        -0.5944, -0.2832,  0.2211, -0.2283, -0.5389, -0.4783, -0.6836, -0.0083,
        -0.1152, -0.3841, -0.4518, -0.2421, -0.4633,  0.3448, -0.4261, -0.1361,
         0.4037, -0.5376,  0.2305, -0.6696,  0.5576, -0.4365, -0.5763])), ('layer_out.weight', tensor([[-0.4868, -0.4487, -0.2316, -0.0912, -0.5542,  0.3977, -0.1459,  0.4104,
         -0.5172,  0.2482, -0.1622,  0.3952, -0.4834,  0.3853, -0.4097,  0.0196,
         -0.0305, 

In [21]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('layer_1.weight', tensor([[-0.6777, -0.6924, -0.5095,  ..., -0.6455, -0.6708, -0.4534],
        [-0.6215, -0.4037, -0.5063,  ..., -0.5613, -0.4609, -0.4414],
        [-0.3246,  0.2038,  0.0809,  ...,  0.3675,  0.1745,  0.1327],
        ...,
        [ 0.0979,  0.3964,  0.3616,  ...,  0.3854,  0.4520,  0.3914],
        [-0.6369, -0.5278, -0.6016,  ..., -0.4705, -0.7159, -0.5438],
        [-0.4271, -0.5144, -0.6358,  ..., -0.5577, -0.6738, -0.5335]])), ('layer_1.bias', tensor([-0.5892, -0.5148,  0.3022,  0.0712, -0.5155, -0.4339,  0.1455, -0.2671,
        -0.5944, -0.2842,  0.2178, -0.2296, -0.5389, -0.4755, -0.6836, -0.0050,
        -0.1177, -0.3843, -0.4518, -0.2443, -0.4633,  0.3438, -0.4261, -0.1372,
         0.4017, -0.5376,  0.2293, -0.6696,  0.5556, -0.4365, -0.5763])), ('layer_out.weight', tensor([[-0.4868, -0.4487, -0.2310, -0.0909, -0.5542,  0.4023, -0.1473,  0.4070,
         -0.5172,  0.2608, -0.1606,  0.3963, -0.4834,  0.3832, -0.4101,  0.0158,
         -0.0277, 

In [22]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [23]:
avg_updates = OrderedDict()
avg_updates["layer_1.weight"] = (
    remote_model1_updates["layer_1.weight"] + remote_model2_updates["layer_1.weight"]
) / 2
avg_updates["layer_1.bias"] = (
    remote_model1_updates["layer_1.bias"] + remote_model2_updates["layer_1.bias"]
) / 2
avg_updates["layer_out.weight"] = (
    remote_model1_updates["layer_out.weight"] + remote_model2_updates["layer_out.weight"]
) / 2
avg_updates["layer_out.bias"] = (
    remote_model1_updates["layer_out.bias"] + remote_model2_updates["layer_out.bias"]
) / 2

print(avg_updates)

OrderedDict([('layer_1.weight', tensor([[-0.6777, -0.6924, -0.5095,  ..., -0.3837, -0.6677, -0.4534],
        [-0.6215, -0.4037, -0.5063,  ..., -0.5613, -0.4609, -0.4414],
        [-0.3267,  0.1950,  0.0731,  ...,  0.3733,  0.2080,  0.1625],
        ...,
        [ 0.0982,  0.3934,  0.3599,  ...,  0.3881,  0.4687,  0.4043],
        [-0.6369, -0.5278, -0.6016,  ..., -0.4705, -0.7159, -0.5438],
        [-0.4271, -0.5144, -0.6358,  ..., -0.5577, -0.6738, -0.5335]])), ('layer_1.bias', tensor([-0.5892, -0.5148,  0.3021,  0.0708, -0.5155, -0.4353,  0.1446, -0.2637,
        -0.5944, -0.2837,  0.2194, -0.2289, -0.5389, -0.4769, -0.6836, -0.0067,
        -0.1164, -0.3842, -0.4518, -0.2432, -0.4633,  0.3443, -0.4261, -0.1366,
         0.4027, -0.5376,  0.2299, -0.6696,  0.5566, -0.4365, -0.5763])), ('layer_out.weight', tensor([[-0.4868, -0.4487, -0.2313, -0.0911, -0.5542,  0.4000, -0.1466,  0.4087,
         -0.5172,  0.2545, -0.1614,  0.3958, -0.4834,  0.3842, -0.4099,  0.0177,
         -0.0291, 

### Load aggregated weights

In [24]:
combined_model = SyNet(torch)

In [25]:
combined_model.load_state_dict(avg_updates)

In [26]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [28]:
import pandas as pd

In [29]:
df_X_val_0 = pd.read_csv('ETL/X_val_0.csv')
df_y_val_0 = pd.read_csv('ETL/y_val_0.csv')
X_val_0_np = df_X_val_0.to_numpy()
y_val_0_np = df_y_val_0.to_numpy()
X_val_0 = th.from_numpy(X_val_0_np).float()
y_val_0 = th.from_numpy(y_val_0_np).float()

In [30]:
preds = []
with torch.no_grad():
    for i in range(len(X_val_0)):
        sample = X_val_0[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {y_val_0[i].item()}")
        preds.append(y_hat)

Prediction: -2.535623073577881 Ground Truth: 0.0
Prediction: -2.4287633895874023 Ground Truth: 0.0
Prediction: -2.4668893814086914 Ground Truth: 0.0
Prediction: -3.034214973449707 Ground Truth: 0.0
Prediction: -2.7363123893737793 Ground Truth: 0.0
Prediction: -2.4142706394195557 Ground Truth: 0.0
Prediction: -3.06044864654541 Ground Truth: 0.0
Prediction: -3.1696157455444336 Ground Truth: 1.0
Prediction: -2.5026416778564453 Ground Truth: 0.0
Prediction: -3.372182846069336 Ground Truth: 0.0
Prediction: -2.7228236198425293 Ground Truth: 0.0
Prediction: -2.8486056327819824 Ground Truth: 0.0
Prediction: -3.07354736328125 Ground Truth: 0.0
Prediction: -2.7564148902893066 Ground Truth: 2.0
Prediction: -2.996823310852051 Ground Truth: 0.0
Prediction: -2.265094041824341 Ground Truth: 0.0
Prediction: -3.1266884803771973 Ground Truth: 0.0
Prediction: -3.0156240463256836 Ground Truth: 0.0
Prediction: -2.824794292449951 Ground Truth: 0.0
Prediction: -2.470412254333496 Ground Truth: 0.0
Prediction:

In [31]:
preds = []
with torch.no_grad():
    for i in range(len(X_val_0)):
        sample = X_val_0[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {y_val_0[i].item()}")
        preds.append(y_hat)

Prediction: 0.10377955436706543 Ground Truth: 0.0
Prediction: 0.2065390944480896 Ground Truth: 0.0
Prediction: 0.04769541323184967 Ground Truth: 0.0
Prediction: 0.05670624226331711 Ground Truth: 0.0
Prediction: 0.015649370849132538 Ground Truth: 0.0
Prediction: 0.10497912764549255 Ground Truth: 0.0
Prediction: 0.10912627726793289 Ground Truth: 0.0
Prediction: 0.060159120708703995 Ground Truth: 1.0
Prediction: 0.07864871621131897 Ground Truth: 0.0
Prediction: 0.17081981897354126 Ground Truth: 0.0
Prediction: 0.1078055277466774 Ground Truth: 0.0
Prediction: 0.14935016632080078 Ground Truth: 0.0
Prediction: 0.04174213111400604 Ground Truth: 0.0
Prediction: 0.07660459727048874 Ground Truth: 2.0
Prediction: 0.09291401505470276 Ground Truth: 0.0
Prediction: 0.11331566423177719 Ground Truth: 0.0
Prediction: 0.1400744915008545 Ground Truth: 0.0
Prediction: -0.0012541860342025757 Ground Truth: 0.0
Prediction: 0.16790495812892914 Ground Truth: 0.0
Prediction: 0.1692904531955719 Ground Truth: 0.0

## Comparison to classical linear regression on centralised data

In [ ]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [ ]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [ ]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [ ]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [ ]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

In [ ]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)